# Key Components and Transaction Flow


## Introduction to Hyperledger Fabric Architecture
### Arianna Groetsema


* [VIDEO](https://youtu.be/nyNUvtsmZNE)

> "Hyperledger Fabric is so unique, because it allows for modular consensus and membership service. This means that algorithms for consensus, identity verification are plug-and-play,resulting in a universal blockchain architecture, that can be applied to most industries or business models."<br><br>
"Channels are another unique feature. They allow transactions to be private between two actors, while still being verified and committed to the blockchain."

## Roles within a Hyperledger Fabric Network

There are three different types of roles within a Hyperledger Fabric network:

### Clients
Clients are applications that act on behalf of a person to propose transactions on the network.

### Peers
Peers maintain the state of the network and a copy of the ledger. There are two different types of peers: **endorsing** and **committing** peers. However, there is an overlap between endorsing and committing peers, in that **endorsing peers are a special kind of committing peers**. All peers commit blocks to the distributed ledger.
- Endorsers **simulate** and endorse transactions
- Committers **verify** endorsements and **validate** transaction results, **prior to committing transactions to the blockchain**.

### Ordering Service 
The ordering service accepts endorsed transactions, orders them into a block, and delivers the blocks to the committing peers.

## How to Reach Consensus

In a distributed ledger system, **consensus** is the process of reaching agreement on the next set of transactions to be added to the ledger. In Hyperledger Fabric, consensus is made up of three distinct steps:
* Transaction endorsement
* Ordering
* Validation and commitment.

These three steps ensure the policies of a network are upheld. We will explore how these steps are implemented by exploring the transaction flow.

## Transaction Flow (Step 1)

Within a Hyperledger Fabric network, transactions start out with client applications sending transaction proposals, or, in other words, proposing a transaction to endorsing peers.

![transaction-flow-fabric-1](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/21431955acd5b7888ca8d393c94deaf8/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Key_Components_-_Transaction_Proposal.png)

**Client applications** are commonly referred to as **applications** or **clients**, and allow people to communicate with the blockchain network. Application developers can leverage the Hyperledger Fabric network through the application SDK.

## Transaction Flow (Step 2)

Each endorsing peer simulates the proposed transaction, without updating the ledger. The endorsing peers will capture the set of **Read** and **Written** data, called **RW Sets**. These RW sets capture what was read from the current world state while simulating the transaction, as well as what would have been written to the world state had the transaction been executed. These RW sets are then signed by the endorsing peer, and returned to the client application to be used in future steps of the transaction flow.

![transaction-flow-fabric-2](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/13e5a6a80c0e150f46d45ec0634b86b8/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Transaction_flow_step_2.png)

### Endorsing peers must hold smart contracts in order to simulate the transaction proposals.

## Transaction Endorsement

A transaction endorsement is **a signed response to the results of the simulated transaction**. The method of transaction endorsements depends on the **endorsement policy which is specified when the chaincode is deployed**.
* An example of an endorsement policy would be "the majority of the endorsing peers must endorse the transaction". Since an endorsement policy is specified for a specific chaincode, different channels can have different endorsement policies.

## Transaction Flow (Step 3)

The application then submits the endorsed transaction and the RW sets to the ordering service. Ordering happens across the network, in parallel with endorsed transactions and RW sets submitted by other applications.

![transaction-flow-fabric-3](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/b6e7b13624d1cff4152e2c223538c355/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Transaction_flow_step_3.png)

## Transaction Flow (Step 4)

The ordering service takes the endorsed transactions and RW sets, orders this information into a block, and delivers the block to all committing peers.

![transaction-flow-fabric-4](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/eeb54ce57f8a6018443e22f34b3ebad9/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Transaction_Flow_Step_4.png)

The **ordering service**, which is made up of a cluster of orderers, does not process transactions, smart contracts, or maintains the shared ledger.<br>

The ordering service
* accepts the endorsed transactions 
* and specifies the order in which those transactions will be committed to the ledger

The Fabric v1.0 architecture has been designed such that the specific implementation of 'ordering' (Solo, Kafka, BFT) becomes a pluggable component. The default ordering service for Hyperledger Fabric is **Kafka**. Therefore, the ordering service is a **modular component of Hyperledger Fabric**.

## Ordering Service
### Chris Ferris

> "if we instead make that in an independent service, and apply the fault tolerance to the ordering service itself, we can actually get a significant improvement in performance and throughput of the overall system."<br><br>
"And so, we've implemented, to date, two ordering services. One is called SOLO - it's really a toy; I mean, it's intended to be used for development purposes, or initial testing of new functions, and so forth. And then, another one is based on an implementation of Kafka."<br><br>
"And, over time, as we go forward, we plan on introducing various forms of fault tolerance too to that ordering service. (RAFT-crash fault tolerant, Simplified Byzantine Fault Tolerance, etc)"

## Ordering (Part I)

*Transactions within a timeframe are sorted into a block and are committed in sequential order.*<br>

**In a blockchain network, transactions have to be written to the shared ledger in a consistent order**. The order of transactions has to be established to ensure that the updates to the world state are valid when they are committed to the network. Unlike the Bitcoin blockchain, where ordering occurs through the solving of a cryptographic puzzle, or mining, **Hyperledger Fabric allows the organizations running the network to choose the ordering mechanism that best suits that network**. 

* This modularity and flexibility makes Hyperledger Fabric incredibly advantageous for enterprise applications.

## Ordering (Part II)

Hyperledger Fabric provides three ordering mechanisms: **SOLO, Kafka, and Simplified Byzantine Fault Tolerance (SBFT)**, the latter of which has not yet been implemented in Fabric v1.0.

* **SOLO** is the Hyperledger Fabric ordering mechanism most typically used by developers **experimenting** with Hyperledger Fabric networks. **SOLO involves a single ordering node**.
* **Kafka** is the Hyperledger Fabric ordering mechanism that is **recommended for production use**. This ordering mechanism utilizes Apache Kafka, an open source stream processing platform that provides a unified, high-throughput, low-latency platform for handling real-time data feeds. In this case, the data consists of endorsed transactions and RW sets. **The Kafka mechanism provides a crash fault-tolerant solution to ordering**.
* **SBFT** stands for **Simplified Byzantine Fault Tolerance**. This ordering mechanism is **both crash fault-tolerant and byzantine fault-tolerant**, meaning that 
  * it can reach agreement even in the presence of malicious or faulty nodes. The Hyperledger Fabric community has not yet implemented this mechanism, but it is on their roadmap.

These three ordering mechanisms provide alternate methodologies for agreeing on the order of transactions.

## Transaction Flow (Step 5)

**The committing peer validates the transaction** by checking to make sure that the RW sets still match the current world state. Specifically, that the Read data that existed when the endorsers simulated the transaction is identical to the current world state. When the committing peer validates the transaction, the transaction is written to the ledger, and the world state is updated with the Write data from the RW Set.

![transaction-flow-fabric-5](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/b05e5430900cf5e414e307d2f99de088/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Transaction_Flow_Step_5.png)

If the transaction fails, 
* that is, if the **committing peer finds that the RW set does not match the current world state**, 
* the **transaction ordered into a block will still be included in that block**, but it will be **marked as invalid**, 
* and the world state will **not be updated**.

Committing peers are responsible for **adding blocks of transactions to the shared ledger and updating the world state**. 
* They may hold smart contracts, but it is not a requirement.

## Transaction Flow (Step 6)

Lastly, the committing peers **asynchronously notify the client application of the success or failure of the transaction**. 
* Applications will be **notified by each committing peer**.

![transaction-flow-fabric-6](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/ba380b73a55eff97c85da3abdc1d86e8/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Transaction_Flow_Step_6.png)



## Identify Verification

In addition to the **multitude of endorsement, validity, and versioning checks** that take place, there are also **ongoing identity verifications happening during each step of the transaction flow**.<br>
* Access control lists are implemented on the **hierarchical layers of the network (from the ordering service down to channels)**, 
* and payloads are repeatedly signed, verified, and authenticated as a transaction proposal passes through the different architectural components.

## Transaction Flow Summary

### It is important to note that the state of the network is maintained by peers, and not by the ordering service or the client. 

Normally, you will design your system such that different machines in the network play different roles. That is, machines that are part of the ordering service should not be set up to also endorse or commit transactions, and vice versa. 
* However, there is **an overlap between endorsing and committing peers** on the system. 
  * Endorsing peers must have access to and hold smart contracts, in addition to fulfilling the role of a committing peer. **Endorsing peers do commit blocks, but committing peers do not endorse transactions**.

### Flow Sequence
1. Endorsing peers verify the client signature, and execute a chaincode function to simulate the transaction. 
  * The **output is the chaincode results, a set of key/value versions** that were read in the chaincode (Read set), and the set of keys/values that were **written by the chaincode**. 
2. The proposal response gets sent back to the client, along with an endorsement signature.
3. These proposal responses are sent to the orderer to be ordered. 
4. The orderer then orders the transactions into a block, which it forwards to the endorsing and committing peers. 
5. The RW sets are used to verify that the transactions are still valid before the content of the ledger and world state is updated. 
6. Finally, the peers asynchronously notify the client application of the success or failure of the transaction.

## Channels

### Channels allow organizations to utilize the same network, while maintaining separation between multiple blockchains. 

Only the members of the channel on which the transaction was performed can see the specifics of the transaction. In other words, **channels partition the network in order to allow transaction visibility for stakeholders only**. This mechanism works by delegating transactions to different ledgers. Only the members of the channel are involved in consensus, while other members of the network do not see the transactions on the channel.

![fabric-channels](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/b23a6aaaa627620a0ab161c556ff87b3/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/Key_Components_Channels.png)

The diagram above shows three distinct channels -- blue, orange, and grey. **Each channel has its own application, ledger, and peers**.<br>
* Peers can belong to multiple networks or channels. 
* Peers that do participate in multiple channels simulate and commit transactions to different ledgers. 
* The ordering service is the same across any network or channel.

A few things to remember:
* The network setup allows for the creation of channels.
* The same chaincode logic can be applied to multiple channels.
* A given user can participate in multiple channels.

## State Database

### The current state data represents the latest values for all assets in the ledger. 
Since the current state represents all the committed transactions on the channel, it is sometimes referred to as world state.<br>

Chaincode invocations execute transactions against the current state data. To make these chaincode interactions extremely efficient, the latest key/value pairs for each asset are stored in a state database. **The state database is simply an indexed view into the chain’s committed transactions**. It can therefore be regenerated from the chain at any time. The state database will **automatically get recovered (or generated, if needed) upon peer startup, before new transactions are accepted**. The default state database, **LevelDB**, can be replaced with **CouchDB**.<br>
* LevelDB is the **default key/value** state database for Hyperledger Fabric, and simply stores key/value pairs.
* CouchDB is an alternative to LevelDB. Unlike LevelDB, CouchDB stores **JSON** objects. CouchDB is **unique** in that it supports 
  * keyed
  * composite
  * key range
  * full data-rich queries

Hyperledger Fabric’s LevelDB and CouchDB are very similar in their structure and function. 
* Both LevelDB and CouchDB support core chaincode operations, such as getting and setting key assets, and querying based on these keys
* With both, keys can be queried by range, and composite keys can be modeled to enable equivalence queries against multiple parameters

But, as a JSON document store, CouchDB additionally enables **rich query against the chaincode data**, when chaincode values (e.g. assets) are modeled as **JSON** data.

![fabric-state-database](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/9fa87a2726077cff05169f85584224ac/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/State_Database.png)

## Smart Contracts

As a reminder, smart contracts are computer programs that contain logic to execute transactions and modify the state of the assets stored within the ledger. 
### Hyperledger Fabric smart contracts are called chaincode and are written in Go.

The chaincode serves as the **business logic** for a Hyperledger Fabric network, in that the chaincode directs **how you manipulate assets within the network**. We will discuss more about chaincode in the Understanding Chaincode section.

## Membership Service Provider (MSP)

The membership service provider, or MSP, is a component that defines the rules in which identities are validated, authenticated, and allowed access to a network.<br>

The MSP manages **user IDs** and authenticates clients who want to join the network. This includes **providing credentials** for these clients to propose transactions. The MSP makes use of a **Certificate Authority, which is a pluggable interface that verifies and revokes user certificates upon confirmed identity**.<br>

The default interface used for the MSP is the **Fabric-CA API**, however, organizations **can implement an External Certificate Authority of their choice**. This is another feature of Hyperledger Fabric that is modular. Hyperledger Fabric supports many credential architectures, which allows for many types of External Certificate Authority interfaces to be used. 
* As a result, a single Hyperledger Fabric network can be controlled by **multiple MSPs**, where each organization brings their favorite.

## What Does the MSP do?

To start, users are authenticated using a certificate authority. The certificate authority **identifies the application, peer, endorser, and orderer identities, and verifies these credentials**. A signature is **generated through the use of a Signing Algorithm and a Signature Verification Algorithm**.<br>

Specifically, generating a signature starts with a Signing Algorithm, which utilizes the credentials of the entities associated with their respective identities, and outputs an endorsement.<br>

1. A signature is generated, which is a byte array that is bound to a specific identity. 
2. Next, the Signature Verification Algorithm takes the identity, endorsement, and signature as inputs, and outputs 'accept' if the signature byte array corresponds with a valid signature for the inputted endorsement, or outputs 'reject' if not. 
  * If the output is 'accept', the user can see the transactions in the network and perform transactions with other actors in the network. 
  * If the output is 'reject', the user has not been properly authenticated, and is not able to submit transactions to the network, or view any previous transactions.

![fabric-msp](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/2fe3f7dc2fa52699a96ef7948432113b/asset-v1:LinuxFoundationX+LFS171x+3T2017+type@asset+block/The_role_of_membership_service_provider.jpg)


## Fabric-Certificate Authority

In general, Certificate Authorities manage enrollment certificates for a permissioned blockchain. 
### Fabric-CA is the default certificate authority for Hyperledger Fabric, and handles the registration of user identities. 

The Fabric-CA certificate authority is in charge of issuing and revoking Enrollment Certificates (E-Certs). The current implementation of Fabric-CA only issues E-Certs, which supply long term identity certificates. E-Certs, which are issued by the Enrollment Certificate Authority (E-CA), assign peers their identity and give them permission to join the network and submit transactions.